In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [8]:
library(EBImage)

dapi_path <- "/home/phillip/documents/lysozyme/lysozyme images/Jej LYZ/G3/AG3FB - 3_DAPI.tif"
rfp_path  <- "/home/phillip/documents/lysozyme/lysozyme images/Jej LYZ/G3/AG3FB - 3_RFP.tif"


load_single_channel <- function(path, channel = c("auto", "red", "green", "blue")) {
    channel <- match.arg(channel)

    img <- readImage(path)               # EBImage Image object
    arr <- as.array(img)
    dims <- dim(arr)

    if (length(dims) == 2L) {
        plane <- img                      # already a single-channel Image
    } else {
        # auto-select the populated plane
        if (channel == "auto") {
            totals <- apply(arr, 3, sum)
            idx <- which.max(totals)         # highest-energy channel
        } else {
            idx <- switch(channel, red = 1L, green = 2L, blue = 3L)
        }
        plane <- img[,,idx, drop = TRUE]     # single channel
    }

    storage.mode(plane) <- "double"

    # normalize to [0,1] if needed
    mx <- max(plane, na.rm = TRUE)
    if (mx > 1) plane <- plane / mx

    # ensure it's an EBImage Image stored explicitly as grayscale
    plane_img <- Image(plane, colormode = "Grayscale")
    colorMode(plane_img) <- 0L

    plane_img
}

blue = dapi_mat = load_single_channel(dapi_path, channel = "blue")
red  = load_single_channel(rfp_path,  channel = "red")







taking these images: dapi_path <- "/home/phillip/documents/lysozyme/lysozyme images/Jej LYZ/G3/AG3FB - 3_DAPI.tif"
rfp_path  <- "/home/phillip/documents/lysozyme/lysozyme images/Jej LYZ/G3/AG3FB - 3_RFP.tif"

import them and convert the follwoign to R, but using imager/ebi and use pipes to keep it clean



red = disp[..., 0].astype(np.float32)
blue = disp[..., 2].astype(np.float32)

diff_r = red > blue #mask_r_erosion swapped, bc unneded


diff_r = morphology.binary_erosion(diff_r, footprint=np.ones((3, 3)))
diff_r = morphology.remove_small_objects(diff_r, min_size=100)
mask_r_dilation = np.maximum(blue, red)
mask_r_erosion = np.minimum(blue, red)
mask_gt_red = blue > 2*red
erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))
mask_u8 = (mask_gt_red.astype(np.uint8) * 255)
mask_eroded_u8 = cv2.erode(mask_u8, erosion_kernel, iterations=2)
mask_gt_red_eroded = mask_eroded_u8.astype(bool)
combined_labels = np.zeros_like(diff_r, dtype=int)
combined_labels[mask_gt_red_eroded] = 2
combined_labels[diff_r] = 1
expanded_labels = expand_labels(combined_labels, distance=100)


# Markers from diff_r (exact notebook logic)
labeled_diff_r, _ = ndi_label(diff_r != 0)

reworked = np.zeros_like(expanded_labels, dtype=np.int32)
reworked[expanded_labels == 2] = 1  # entire class 2 region => marker 1
mask_copy = (expanded_labels != 2) & (labeled_diff_r != 0) # expanded_labels != 2 is background 0 and crypts 1, labeled_diff_r != 0 is labeled crypts from diff_r
reworked[mask_copy] = labeled_diff_r[mask_copy] + 1
mask_ws = expanded_labels > 0
elevation = (
    minmax01(distance_transform_edt(combined_labels == 2))
    - minmax01(distance_transform_edt(combined_labels == 1))
)


# Apply watershed (exact notebook logic)
ws_labels = watershed(elevation, markers=reworked, mask=mask_ws)
